# Materials Calculator: MVP

## Instructions:

1. Go to the "Items tab" of your Excel Sheet working copy.
2. Save as > File Format > CSV
3. Find the saved file on your computer, and option + right click it to "Copy as Pathname"
4. Run the following cells and insert the pathname when prompted
5. Insert the item quantities, one by one, as prompted
6. Tadaaaa!

In [ ]:
import pandas as pd
import numpy as np

### SPREADSHEET INPUT

path = input("Please insert the path to your csv file:")
costs = pd.read_csv(path)

### CLEANING

costs = costs.dropna(how='all', axis=1)
costs = costs.dropna(how='all', axis=0)

costs["Description"] = costs["Description"].fillna("")
costs['Item Name'] = costs['Item Name'].str.strip()

del costs["Unnamed: 24"]

new_costs = []
           
for cost_name, item_name, description, uom in zip(costs['Cost Name'], costs['Item Name'], costs["Description"], costs["UOM"]):
    
    # Renaming dyes for clarity on colour
    if cost_name == "Dye":
        new_costs.append(str(cost_name + " -" + item_name))
        
    # Replacing question marks with Item Names for Clarity
    elif "?" in cost_name:
        replacement = "for" + str(item_name)
        new_costs.append(cost_name.replace('?"', replacement))
        
    # Adding the material kind to YKK zips
    elif "YKK" in cost_name:
        new_costs.append(str(cost_name + " (" + description + ")"))
    
    # Adding the size to the Sublimation Dyes
    elif cost_name == "Sublimation Dye":
        new_costs.append(str(cost_name + " (" + uom + ")"))
    
    else:
        new_costs.append(cost_name)
    
costs["costs"] = new_costs

new_UOM = []

for uom in costs["UOM"]:
    if "A" in uom:
        new_UOM.append("pcs")
    else:
        new_UOM.append(uom)

costs["UOM"] = new_UOM

material_costs = costs[costs["Cost Label"] == "Materials"]

material_costs = material_costs.drop(["C-Code", "Cost Name", "I-Code", "Unit Cost", "Cost Label", "Cost Sublabel", "Description"], axis=1)

cost_per = []

for cost in material_costs["Cost Per"]:
    if "-" in cost:
        cost_per.append("0")
    elif "£" in cost:
        cost_per.append(cost.replace("£", ""))
    else:
        cost_per.append(cost)

cost_per = [float(i) for i in cost_per]
material_costs["Cost Per"] = cost_per

del new_costs, new_UOM, cost_per

material_costs = material_costs.rename(columns={"Yield": "Units", "Cost Per": "Cost Per Unit", "costs" : "Material"})

costs = pd.DataFrame(material_costs)

### USER INPUT - ORDER QUANTITY

item_names = {}

for item_name in costs["Item Name"].unique():
    item_names[item_name] = 0
    
for key, value in item_names.items():
    item_names[key] = int(input("How many " + key + "s?"))
    
### CALCULATIONS

costs['Quantity'] = costs['Item Name'].map(item_names)

material_costs = costs.copy()

del material_costs["Item Name"]

material_costs["Total Units"] = material_costs["Units"] * material_costs["Quantity"]

material_costs["Total Cost"] = material_costs["Total Units"] * material_costs["Cost Per Unit"]

material_costs[['Material', 'Total Units', 'UOM', 'Total Cost']]


### OUTPUT

print("Your Quantity Request:")

print("\n")

for key, value in item_names.items():
    print(str(value) + " " + str(key) + "s")
    
print("\n")

print("This order will cost you", round(material_costs["Total Cost"].sum(),2), "in total")
print("\n")
print("Your price breakdown is:")

output_costs = material_costs[['Material', 'Total Units', 'UOM', 'Total Cost']].groupby('Material', as_index=False).sum(numeric_only = True).round(2)

mat_and_uom = {}

for material, uom in zip(material_costs['Material'], material_costs['UOM']):
    if material not in mat_and_uom.keys():
        mat_and_uom[material] = uom

output_costs['Units of Measure'] = output_costs['Material'].map(mat_and_uom)
output_costs = output_costs[["Material", "Total Units", "Units of Measure", "Total Cost"]]

output_costs